In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from textblob import TextBlob
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF  # Matrix Factorization
from pyvi import ViTokenizer
from pyvi import ViUtils
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer


In [2]:
fpt_df = pd.read_csv('new_comments_fpt.csv', encoding='utf-8')
tgdd_df = pd.read_csv('new_comments_tgdd.csv', encoding='utf-8')

In [3]:
fpt_df

,phone_name,user_name,used_to_buy,content,star_rating
0,Samsung Galaxy S22 5G 128GB,Đức Nam,yes,"nhận máy và xài, công nhận giá quá tốt",4
1,Samsung Galaxy S22 5G 128GB,Nguyễn Phương,yes,"dt mượt, cấu hình ok, nv nhiệt tình, dt cầm ch...",5
2,Samsung Galaxy S22 5G 128GB,Phạm Văn Thanh,yes,ko có gì chê cả. Nên mua nhé cả nhà. Mình chỉ ...,4
3,Samsung Galaxy S22 5G 128GB,Ngọc Trinh,yes,"Muot ma, sac pin nhanh, kieu dang dep, thoi lu...",4
4,Samsung Galaxy S22 5G 128GB,Lâm Quang Bình,yes,"Thiết kế sexy, cầm nắm đã tay, cảm giác mềm mạ...",5
...,...,...,...,...,...
10118,Nokia 110 DS 4G,khanh sơn,no,thời lượng pin siêu dài lên đến 18 giờ,5
10119,Nokia 110 DS 4G,Lạc Anh,no,Chất lượng sản phẩm hoàn toàn có thể yên tâm t...,5
10120,Nokia 110 DS 4G,Nguyễn văn quyết,no,"máy nhỏ gọn, pin dùng được lâu, màn hình sắc n...",5
10121,Nokia 110 DS 4G,Nam Duy,no,Sản phẩm rất tốt rất đáng để mua nha mn có rất...,5


In [4]:
tgdd_df

,phone_name,user_name,content,star_rating
0,iPhone 15 Pro Max,Hiệp Nguyễn,Sản phẩm tốt chất lượng nhân viên tư vấn TGDĐ ...,5
1,iPhone 15 Pro Max,Vũ,"Mới mua được 3 hôm,lướt web với fb thôi mà cũn...",5
2,iPhone 15 Pro Max,Cón,Tạm ổn,3
3,iPhone 15 Pro Max,Nguyen Thanh Con,Nhân viên tư vấn thân thiệt và nhiệt tình,5
4,iPhone 15 Pro Max,trần văn toàn,sao pin con này bị sao vậy shop,5
...,...,...,...,...
4178,Mobell M239,Nguyễn Van Hảo,Chưa sạc lần nào cục sạc đã hư,2
4179,Mobell M239,Đoàn Minh Lâm,Mua lại máy đã qua sử dụng nói chung pin cũng ...,3
4180,Mobell M239,Trần Trung Hiếu,Máy phổ thông mà pin khá yếu chỉ được 1 ngày d...,2
4181,Mobell M239,Anh Châu,Ok,5


In [5]:
# Group by star_rating and count the frequency of each comment
comment_counts = tgdd_df.groupby(['star_rating', 'content']).size().reset_index(name='count')

# Find the comment with the highest frequency for each star rating
max_comments = comment_counts.groupby('star_rating').apply(lambda x: x.loc[x['count'].idxmax()])

# Display the result
max_comments['content'][1] = 'tệ'
max_comments['content'][2] = 'Không tốt'
max_comments

C:\Users\admin\AppData\Local\Temp\ipykernel_212\1452591716.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_comments['content'][1] = 'tệ'
C:\Users\admin\AppData\Local\Temp\ipykernel_212\1452591716.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_comments['content'][2] = 'Không tốt'


,star_rating,content,count
star_rating,,,
1,1,tệ,1
2,2,Không tốt,2
3,3,Bình thường,4
4,4,Tốt,63
5,5,Ok,106


In [6]:
for i in range(len(tgdd_df)):
    if(type(tgdd_df['content'][i]) == float):
        tgdd_df['content'][i] = max_comments['content'][tgdd_df['star_rating'][i]]

C:\Users\admin\AppData\Local\Temp\ipykernel_212\2044535501.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tgdd_df['content'][i] = max_comments['content'][tgdd_df['star_rating'][i]]
C:\Users\admin\AppData\Local\Temp\ipykernel_212\2044535501.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tgdd_df['content'][i] = max_comments['content'][tgdd_df['star_rating'][i]]
C:\Users\admin\AppData\Local\Temp\ipykernel_212\2044535501.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [7]:
user_comments_fpt = fpt_df['content'].values
user_comments_tgdd = tgdd_df['content'].values
user_comments = np.concatenate((user_comments_fpt, user_comments_tgdd))
len(user_comments)

14306

In [8]:
user_star_fpt = fpt_df['star_rating'].values
user_star_tgdd = tgdd_df['star_rating'].values
user_star = np.concatenate((user_star_fpt, user_star_tgdd))
len(user_star)

14306

In [9]:
phone_name_fpt = fpt_df['phone_name'].values
phone_name_tgdd = tgdd_df['phone_name'].values
phone_name = np.concatenate((phone_name_fpt, phone_name_tgdd))
len(phone_name)

14306

In [10]:
def preprocess_text_vi(text, max_length=256):
    if type(text) == float: 
        text = str(text)
    stopwords = set([
        "là", "của", "ở", "và", "hay", "cùng", "một", "nhiều", "rất", "này", 
        "nhưng", "với", "vì", "đã", "được", "có", "không", "thì", "cho", "để"
    ])
    
    text_pre = list(tf.keras.preprocessing.text.text_to_word_sequence(text))
    
    text_pre_no_stopwords = [word for word in text_pre if word.lower() not in stopwords]

    text_pre_no_stopwords = " ".join(text_pre_no_stopwords)
    
    text_pre_accent = ViTokenizer.tokenize(text_pre_no_stopwords)

    if len(text_pre_accent) > max_length:
      text_pre_accent = text_pre_accent[:max_length]
    
    return text_pre_accent

def evaluate_review(phone_name_list, user_comments_list, user_star_list):
  star_1_5 = 0.25
  star_2_4 = 0.125
  star_3 = 0.25
  res = []
  model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")
  tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

  for i in range(len(user_comments_list)):
    # Preprocess text (optional punctuation removal commented out)
    text = preprocess_text_vi(user_comments_list[i])

    if type(text) == float: 
        text = str(text)

    # Convert text to input IDs
    input_ids = torch.tensor([tokenizer.encode(text)])

    # Perform sentiment analysis with phoBERT (get logits)
    with torch.no_grad():
      out = model(input_ids)
      logits = out.logits  # Get logits before softmax

    # Get maximum probability for sentiment score
    sentiment_score = logits.softmax(dim=-1).tolist()[0]

    # Re-evaluate sentiment score
    if(user_star_list[i] == 1):
      sentiment_score[0] += star_1_5
    elif(user_star_list[i] == 2):
      sentiment_score[0] += star_2_4
    elif(user_star_list[i] == 3):
      sentiment_score[2] += star_3
    elif(user_star_list[i] == 4):
      sentiment_score[1] += star_2_4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
    elif(user_star_list[i] == 5):
      sentiment_score[1] += star_1_5

    adjusted_score = float(-sentiment_score[0] * 2 + sentiment_score[1] * 2 + sentiment_score[2]) / 5

    res.append((phone_name_list[i], user_comments_list[i], adjusted_score, user_star_list[i]))
    with open("sentiment_results.pickle", "wb") as f:
      pickle.dump(res, f)
  return res

In [11]:
results = evaluate_review(phone_name, user_comments, user_star)

In [24]:
for phone_name, comment, score, rating in results:
  print(f"Phone: {phone_name}, Comment: {comment}, Sentiment Score: {score}, Rating: {rating}")

Phone: Samsung Galaxy S22 5G 128GB, Comment: nhận máy và xài, công nhận giá quá tốt, Sentiment Score: 0.4474706262350082, Rating: 4
Phone: Samsung Galaxy S22 5G 128GB, Comment: dt mượt, cấu hình ok, nv nhiệt tình, dt cầm chắc tay, vừa vặn, Sentiment Score: 0.49758941256441175, Rating: 5
Phone: Samsung Galaxy S22 5G 128GB, Comment: ko có gì chê cả. Nên mua nhé cả nhà. Mình chỉ tiếc cái là ko có lỗ tai nghe 3.5mm thôi., Sentiment Score: 0.44684866834431886, Rating: 4
Phone: Samsung Galaxy S22 5G 128GB, Comment: Muot ma, sac pin nhanh, kieu dang dep, thoi luong pin cung tot a⁰, Sentiment Score: 0.44739486165344716, Rating: 4
Phone: Samsung Galaxy S22 5G 128GB, Comment: Thiết kế sexy, cầm nắm đã tay, cảm giác mềm mại như làn da thiếu nữ vậy Màn hình đẹp, pin trâu, hiệu năng ổn định, Sentiment Score: 0.4975123685784638, Rating: 5
Phone: Samsung Galaxy S22 5G 128GB, Comment: Cảm ơn shop nhiều, đã nhận máy với giá người nhà, ưu đãi lắm, Sentiment Score: 0.4471034579910338, Rating: 4
Phone: Sa

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from textblob import TextBlob
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF  # Matrix Factorization
from pyvi import ViTokenizer
from pyvi import ViUtils
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

In [14]:
with open("pre_sentiment_results.pkl", 'rb') as file:
    results = pickle.load(file)

In [15]:
results

[('Samsung Galaxy S22 5G 128GB',
  'nhận máy và xài, công nhận giá quá tốt',
  0.4474706262350082,
  4),
 ('Samsung Galaxy S22 5G 128GB',
  'dt mượt, cấu hình ok, nv nhiệt tình, dt cầm chắc tay, vừa vặn',
  0.49758941256441175,
  5),
 ('Samsung Galaxy S22 5G 128GB',
  'ko có gì chê cả. Nên mua nhé cả nhà. Mình chỉ tiếc cái là ko có lỗ tai nghe 3.5mm thôi.',
  0.44684866834431886,
  4),
 ('Samsung Galaxy S22 5G 128GB',
  'Muot ma, sac pin nhanh, kieu dang dep, thoi luong pin cung tot a⁰',
  0.44739486165344716,
  4),
 ('Samsung Galaxy S22 5G 128GB',
  'Thiết kế sexy, cầm nắm đã tay, cảm giác mềm mại như làn da thiếu nữ vậy Màn hình đẹp, pin trâu, hiệu năng ổn định',
  0.4975123685784638,
  5),
 ('Samsung Galaxy S22 5G 128GB',
  'Cảm ơn shop nhiều, đã nhận máy với giá người nhà, ưu đãi lắm',
  0.4471034579910338,
  4),
 ('Samsung Galaxy S22 5G 128GB',
  'Thích cái kho ứng dụng của samsung, cực đa dạng và rất dễ dùng',
  0.4474903336260468,
  4),
 ('Samsung Galaxy S22 5G 128GB',
  'Thấy x

In [16]:
def create_features(phones_name, review_texts, adjusted_scores, star_ratings):
    # TF-IDF vectorizer for review text
    vectorizer = TfidfVectorizer(max_features=1000)
    review_features = vectorizer.fit_transform(review_texts)

    # Convert phones_name to DataFrame
    phones_df = pd.DataFrame(phones_name, columns=['phone_name'])

    # Combine phone features and review features
    features = pd.concat([phones_df, pd.DataFrame(adjusted_scores), pd.DataFrame(star_ratings), pd.DataFrame(review_texts)], axis=1)
    features.columns=['phone_name', 'sentiment_score', 'stars', 'content']
    return features, review_features

phones_name, review_texts, adjusted_scores, star_ratings = zip(*results)  # Unpack results
features, review_features = create_features(phones_name, review_texts, adjusted_scores, star_ratings)
features

,phone_name,sentiment_score,stars,content
0,Samsung Galaxy S22 5G 128GB,0.447471,4,"nhận máy và xài, công nhận giá quá tốt"
1,Samsung Galaxy S22 5G 128GB,0.497589,5,"dt mượt, cấu hình ok, nv nhiệt tình, dt cầm ch..."
2,Samsung Galaxy S22 5G 128GB,0.446849,4,ko có gì chê cả. Nên mua nhé cả nhà. Mình chỉ ...
3,Samsung Galaxy S22 5G 128GB,0.447395,4,"Muot ma, sac pin nhanh, kieu dang dep, thoi lu..."
4,Samsung Galaxy S22 5G 128GB,0.497512,5,"Thiết kế sexy, cầm nắm đã tay, cảm giác mềm mạ..."
...,...,...,...,...
14301,Mobell M239,-0.441264,2,Chưa sạc lần nào cục sạc đã hư
14302,Mobell M239,0.232173,3,Mua lại máy đã qua sử dụng nói chung pin cũng ...
14303,Mobell M239,-0.441786,2,Máy phổ thông mà pin khá yếu chỉ được 1 ngày d...
14304,Mobell M239,0.489696,5,Ok


In [17]:
average_sentiment_by_phone = features.groupby('phone_name')['sentiment_score'].mean()
df_avg_sentiment_by_phone = pd.DataFrame(average_sentiment_by_phone).reset_index()

In [18]:
min_sentiment_index = df_avg_sentiment_by_phone['sentiment_score'].idxmin()

# Access the row using .loc[]
row_with_min_sentiment = df_avg_sentiment_by_phone.loc[min_sentiment_index]
row_with_min_sentiment

phone_name         Xiaomi Redmi Note 12 Pro 128GB
sentiment_score                         -0.183977
Name: 115, dtype: object

In [19]:
max_sentiment_index = df_avg_sentiment_by_phone['sentiment_score'].idxmax()

# Access the row using .loc[]
row_with_max_sentiment = df_avg_sentiment_by_phone.loc[max_sentiment_index]
row_with_max_sentiment

phone_name         Samsung Galaxy S24 5G 256GB
sentiment_score                       0.497452
Name: 86, dtype: object

In [20]:
test = []
for phone_name in features['phone_name'].values:
  if '15 Pro Max' in phone_name and phone_name not in test:
    test.append(phone_name)
test

['iPhone 15 Pro Max 256GB', 'iPhone 15 Pro Max']

In [21]:
for index, row in df_avg_sentiment_by_phone.iterrows():
    phone_name = row['phone_name']
    if phone_name.endswith('GB'):
        parts = phone_name.rsplit(' ', 1)
        if len(parts) > 1:
            df_avg_sentiment_by_phone.at[index, 'phone_name'] = parts[0]

In [22]:
df_avg_sentiment_by_phone[df_avg_sentiment_by_phone['sentiment_score'] < 0]

,phone_name,sentiment_score
22,Nokia 105,-0.106599
73,Samsung Galaxy A25 5G,-0.033928
79,Samsung Galaxy M54 5G,-0.031813
88,Samsung Galaxy S24 Ultra 5G,-0.060948
115,Xiaomi Redmi Note 12 Pro,-0.183977


In [23]:
df_avg_sentiment_by_phone.to_csv('final_sentiment_results.csv', index=False)

In [24]:
test

['iPhone 15 Pro Max 256GB', 'iPhone 15 Pro Max']